<a href="https://colab.research.google.com/github/CINDYGHM77/BRBK_M12025/blob/main/MAT-EXAMEN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exámen 2 del módulo de matemáticas
## Cindy Guadalupe Hernández Morales

In [15]:
import numpy as np
import io, zipfile, requests, pandas as pd
from PIL import Image
from IPython.display import display
from sklearn.decomposition import PCA, TruncatedSVD

#### **Ejercicio 1**
#### Sin utilizar Numpy defina una función en Python para multiplicar dos matrices de tamañoarbitrario.

In [4]:
## Función que multiplica dos matrices
def multiplicar_matrices(A, B):
    # Verifica que se puedan multiplicar (número de columnas de A = número de filas de B)
    if len(A[0]) != len(B):
        raise ValueError("El número de columnas de A debe ser igual al número de filas de B")

    # Crea la matriz resultado con ceros
    resultado = [[0 for _ in range(len(B[0]))] for _ in range(len(A))]

    # Multiplicación
    for i in range(len(A)):
        for j in range(len(B[0])):
            print("-"*5)
            for k in range(len(B)):
                print(f"A[{i}][{k}] {A[i][k]} * B[{k}][{j}] {B[k][j]}")
                resultado[i][j] += A[i][k] * B[k][j]

    return resultado

In [3]:
# Definición de matrices
A = [[1, 2, 3],
     [4, 5, 6]]

B = [[7, 8],
     [9, 10],
     [11, 12]]

resultado = multiplicar_matrices(A, B)
print("-"*5)
print(resultado) # El resultado es una matriz de 2 x 2

-----
A[0][0] 1 * B[0][0] 7
A[0][1] 2 * B[1][0] 9
A[0][2] 3 * B[2][0] 11
-----
A[0][0] 1 * B[0][1] 8
A[0][1] 2 * B[1][1] 10
A[0][2] 3 * B[2][1] 12
-----
A[1][0] 4 * B[0][0] 7
A[1][1] 5 * B[1][0] 9
A[1][2] 6 * B[2][0] 11
-----
A[1][0] 4 * B[0][1] 8
A[1][1] 5 * B[1][1] 10
A[1][2] 6 * B[2][1] 12
-----
[[58, 64], [139, 154]]


#### **Ejercicio 2**
#### Utilizar la descomposición en valores singulares para resolver alguno de los sistemas de ecuaciones de la primera semana.

$$
\begin{cases}
y - x + 1 = 0\\
0y + 3x - 0 = 0
\end{cases}
$$

$$
\begin{cases}
-x + y = -1\\
+3x + 0y = 0
\end{cases}
$$


In [7]:
# Despomposición de Vectores Singulares (SVD)
A = np.array([[-1, 1],
              [3, 0]], dtype=float)
b = np.array([-1, 0], dtype=float)

# Descomposición SVD
U, S, VT = np.linalg.svd(A)

# Invertimos los valores singulares
S_inv = np.diag(1 / S)

# Construimos la inversa manualmente
A_inv = VT.T @ S_inv @ U.T

# Resolvemos el sistema
x = A_inv @ b
print("Solución usando SVD:", x)

Solución usando SVD: [ 5.18669719e-18 -1.00000000e+00]


#### **Ejercicio 3**
#### Utilizando la base de datos MNIST, comparar la reducción de PCA contra la de descomposición en valores singulares. La métrica que utilizaremos será la medida del coseno promedio (al reducir la dimensión) entre las imágenes del mismo dígito.

In [10]:
url = "https://www.interactivechaos.com/data/mnist.zip"

# Descarga el ZIP en memoria
resp = requests.get(url, stream=True)
resp.raise_for_status()

# Abre el ZIP desde memoria
with zipfile.ZipFile(io.BytesIO(resp.content)) as z:
    csv_name = [n for n in z.namelist() if n.lower().endswith(".csv")][0]
    print("CSV dentro del ZIP:", csv_name)

    with z.open(csv_name) as f:
        df = pd.read_csv(f, header=None)

y = df.iloc[:, 0].to_numpy()
X = df.iloc[:, 1:].to_numpy(dtype="float32") / 255.0

print("df:", df.shape, "X:", X.shape, "y:", y.shape)

CSV dentro del ZIP: mnist.csv
df: (60000, 785) X: (60000, 784) y: (60000,)


In [11]:
min(X[10]), max(X[10])

(np.float32(0.0), np.float32(0.99607843))

In [14]:

def ver_par_pil(X, y, idx1, idx2):
    def to_img(x):
        arr = (x.reshape(28, 28) * 255).clip(0, 255).astype(np.uint8)
        return Image.fromarray(arr, mode="L")

    im1, im2 = to_img(X[idx1]), to_img(X[idx2])
    print(f"Primera etiqueta: {y[idx1]}")
    display(im1)
    print(f"Segunda etiqueta: {y[idx2]}")
    display(im2)

ver_par_pil(X, y, 10, 11)


Primera etiqueta: 3


/tmp/ipython-input-214381054.py:4: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  return Image.fromarray(arr, mode="L")


Segunda etiqueta: 5


In [17]:
def mean_cosine_intra_digit_loops(Z, y, max_rows=500):
    eps = 1e-12
    per_digit = {}

    for d in range(10):
        idx = np.where(y == d)[0]
        if idx.size < 2:
            per_digit[d] = np.nan
            continue

        sum_cos, cnt = 0.0, 0
        if max_rows is not None:
            idx = idx[:max_rows]
        for a in range(len(idx)):
            i = idx[a]
            zi = Z[i]
            ni = np.linalg.norm(zi)
            if ni < eps:
                continue
            for b in range(a + 1, len(idx)):
                j = idx[b]
                zj = Z[j]
                nj = np.linalg.norm(zj)
                if nj < eps:
                    continue
                cos_ij = float(np.dot(zi, zj) / (ni * nj))
                sum_cos += cos_ij
                cnt += 1

        per_digit[d] = (sum_cos / cnt) if cnt > 0 else np.nan

    vals = [v for v in per_digit.values() if not np.isnan(v)]
    global_mean = float(np.mean(vals)) if vals else np.nan
    return per_digit, global_mean

In [18]:
# Dimensiones a obtener
k = 50

# PCA
pca = PCA(n_components=k, random_state=0)
Z_pca = pca.fit_transform(X)

# SVD
tsvd = TruncatedSVD(n_components=k, random_state=0)
Z_svd = tsvd.fit_transform(X)

In [19]:
# Comparar la media del conseno entre numeros
per_pca, g_pca = mean_cosine_intra_digit_loops(Z_pca, y, max_rows=2000)
per_svd, g_svd = mean_cosine_intra_digit_loops(Z_svd, y, max_rows=2000)

In [20]:
df = pd.DataFrame({
    "Digito": list(range(10)),
    "PCA": [per_pca[d] for d in range(10)],
    "SVD": [per_svd[d] for d in range(10)],
})
print(df.to_string(index=False, float_format=lambda v: f"{v:.3f}"))
print("Media global PCA:", round(g_pca, 3))
print("Media global mean SVD:", round(g_svd, 3))

 Digito   PCA   SVD
      0 0.336 0.632
      1 0.475 0.632
      2 0.187 0.567
      3 0.211 0.592
      4 0.222 0.561
      5 0.111 0.500
      6 0.266 0.611
      7 0.288 0.584
      8 0.149 0.620
      9 0.192 0.592
Media global PCA: 0.244
Media global mean SVD: 0.589
